The goal of this notebook is to decipher the code ```PEA_SAM_all.py``` and its components for further use and communication.

Here is [a notebook example](https://nsrdb.nrel.gov/api-instructions).

**IMPORTANT**: Make sure that you put this code and the Shift+F5 core Python output from GUI SAM in the SDK folder. Also make sure that the Python and SAM version are correct.

In [16]:
import sys
print (sys.version)

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jun 29 2016, 11:07:13) [MSC v.1500 64 bit (AMD64)]


In [17]:
SDKdir = r"C:/Users/Admin/Downloads/sam-sdk-2016-3-14-r3/languages/python/"

In [18]:
#Import necessary Python libraries
import site
site.addsitedir(SDKdir)
from PySSC import PySSC

import csv
import os
import numpy

#Specify data directories
DataDir = r'C:/Users/Admin/Dropbox/Solar NEM (ERI)/Data manipulation/'

First, let's check out the function called ```simulate```. Here are inputs:

- Scheme: Categorical values 1= Pilot/ No buyback, 2 = Net Metering/ NEM, 3 = Net Billing/ Real-time buyback
- Sensitivity: Categorical values 1 = below retail/ wholesale, 2 = retail/central case, 3 = above retail / percent add-on
- Customer: Categorical values. It can be categorized further based on types:
    - 11 -> ["Res", "Res_TOU"]
    - 20 -> ["SGS", "SGS_TOU"]
    - 30 -> ["MGS"]
    - 40 -> ["LGS", "LGS_BOI"]


In [19]:
Customer = 'Res'
Scheme = 1
Sensitivity = 1
weather_filename = 'THA_ARANYAPRATHET_484620_IW2'
weather_file = DataDir + b'SAM/' + b'THA_weather_data/' + weather_filename + b'/' + weather_filename + b'.EPW'
Region = 16

#result_file1 = 
#result_file2 = 

#New input: assign case if system_size is determined
system_size = 3
#recommend system_size = 3 for ["Res","SGS"] and ["Res_TOU","SGS_TOU"]
#recommend system_size = 100 for ["MGS"]    
#recommend system_size = 1000 for ["LGS", "LGS_BOI"]

The first step in the simulation is to define the rate.

In [20]:
#Basic buyback rate
if Customer in ["Res","SGS"] :
    block1 = 0.083
    block2 = 0.111
    block3 = 0.117
    block_average = (block1 + block2 + block3)/3 #Not using it after all?

if Customer in ["Res_TOU","SGS_TOU"] :
    TOU_retail_1 = 0.066
    TOU_retail_2 = 0.156
    TOU_retail_average = (TOU_retail_1 + TOU_retail_2)/2
    
    TOU_wholesale_1 = 0.057823
    TOU_wholesale_2 = 0.111183
    TOU_wholesale_average = (TOU_wholesale_1 + TOU_wholesale_2)/2
    
elif Customer in ["MGS","LGS", "LGS_BOI"]:
    TOU_retail_1 = 0.066
    TOU_retail_2 = 0.111
    TOU_retail_average = (TOU_retail_1 + TOU_retail_2)/2
    
    TOU_wholesale_1 = 0.057106
    TOU_wholesale_2 = 0.087409
    TOU_wholesale_average = (TOU_wholesale_1 + TOU_wholesale_2)/2
 

In [21]:
#Buyback rate for different customers and sensitivies (Sensitivity: 1 = below retail, 2 = retail, 3 = above retail)
if Customer in ["Res","SGS"] :
    buyback_block_values = [0,                  1/35,               2/35,                    3/35]
if Customer in ["Res_TOU","SGS_TOU","MGS","LGS", "LGS_BOI"]:
    buyback_TOU_net_metering_values = [0, TOU_wholesale_average, TOU_retail_average, TOU_retail_average*1.05]
    buyback_TOU_net_billing_offpeak_values = [0,               1/35,               2/35,               3/35] 
    buyback_TOU_net_billing_peak_values = [0,                1/35,               2/35,                 3/35]

Next, we choose the corresponding load profile based on region and customer type.

In [22]:
#Set load profile based on Customer Type
if Customer in ["Res", "Res_TOU"]:
    Customer_type = b'11'
elif Customer in ["SGS", "SGS_TOU"]:
    Customer_type = b'20'
elif Customer == "MGS":
    Customer_type = b'30'
elif Customer in ["LGS", "LGS_BOI"]:
    Customer_type = b'40' 
    
load_file = DataDir + '/2015/load profile/DATA' + str(Region).encode('utf-8') + '_' + Customer_type + '.csv'

Next, we populate the necessary values for the simulation. Some notes on variables:
- Metering option: metering = 2 for pilot and net-billing, 0 for net-metering
- NEM_sale_rate: price assigned for net-metering scheme, 0 for other schemes.

In [23]:
#Fill in necessary values
if Customer in ["Res","SGS"] :
    install_cost = 1.93*1000*system_size 
    discount_rate = 2.67
    
    ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    
    if Scheme == 1: #pilot
        ur_ec_tou_mat = [None]*3
        ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   0 ]
        ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   0 ]
        ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   0 ]
        metering = 2
        NEM_sale_rate = 0
        
    elif Scheme == 2: #net-metering
        ur_ec_tou_mat = [None]*3
        ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   0 ]
        ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   0 ]
        ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   0 ]
        metering = 0
        NEM_sale_rate = buyback_block_values[Sensitivity]

    elif Scheme == 3: #net-billing
        ur_ec_tou_mat = [None]*3
        ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   buyback_block_values[Sensitivity] ]
        ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   buyback_block_values[Sensitivity] ]
        ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   buyback_block_values[Sensitivity] ]
        metering = 2
        NEM_sale_rate = 0
    
    fixed_charge = 1.0920000076293945
    ur_dc_sched_weekday = ur_ec_sched_weekday
    ur_dc_sched_weekend = ur_dc_sched_weekday
    ur_dc_flat_mat = [None]*12
    ur_dc_flat_mat[0] = [ 0,   1,   168.69999694824219,   3.7980000972747803 ]
    ur_dc_flat_mat[1] = [ 1,   1,   200,                  3.7980000972747803 ]
    ur_dc_flat_mat[2] = [ 2,   1,   178.60000610351562,   3.7980000972747803 ]
    ur_dc_flat_mat[3] = [ 3,   1,   193.1300048828125,    3.7980000972747803 ]
    ur_dc_flat_mat[4] = [ 4,   1,   189.92999267578125,   3.7980000972747803 ]
    ur_dc_flat_mat[5] = [ 5,   1,   184.92999267578125,   3.7980000972747803 ]
    ur_dc_flat_mat[6] = [ 6,   1,   176.6199951171875,    3.7980000972747803 ]
    ur_dc_flat_mat[7] = [ 7,   1,   168.80000305175781,   3.7980000972747803 ]
    ur_dc_flat_mat[8] = [ 8,   1,   177.66000366210938,   3.7980000972747803 ]
    ur_dc_flat_mat[9] = [ 9,   1,   171.44000244140625,   3.7980000972747803 ]
    ur_dc_flat_mat[10] = [ 10,   1,   189.21000671386719,  3.7980000972747803 ]
    ur_dc_flat_mat[11] = [ 11,   1,   177.3800048828125,   3.7980000972747803 ]
    ur_dc_tou_mat = [[ 1,   1,   9e+37,   0 ]]
    dc_enable = 0

elif Customer in ["Res_TOU","SGS_TOU"] :
    install_cost = 1.93*1000*system_size 
    discount_rate = 2.67
    ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
    ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    if Scheme == 1:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 2
        NEM_sale_rate = 0
    elif Scheme == 2:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 0
        NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
    elif Scheme == 3:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,  buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,  buyback_TOU_net_billing_peak_values[Sensitivity] ]
        metering = 2
        NEM_sale_rate = 0

    fixed_charge = 1.0920000076293945
    ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    ur_dc_sched_weekend = ur_dc_sched_weekday 
    ur_dc_flat_mat = [None]*12
    ur_dc_flat_mat[0] = [ 0,   1,   168.69999694824219,   3.7980000972747803 ]
    ur_dc_flat_mat[1] = [ 1,   1,   200,                  3.7980000972747803 ]
    ur_dc_flat_mat[2] = [ 2,   1,   178.60000610351562,   3.7980000972747803 ]
    ur_dc_flat_mat[3] = [ 3,   1,   193.1300048828125,    3.7980000972747803 ]
    ur_dc_flat_mat[4] = [ 4,   1,   189.92999267578125,   3.7980000972747803 ]
    ur_dc_flat_mat[5] = [ 5,   1,   184.92999267578125,   3.7980000972747803 ]
    ur_dc_flat_mat[6] = [ 6,   1,   176.6199951171875,    3.7980000972747803 ] 
    ur_dc_flat_mat[7] = [ 7,   1,   168.80000305175781,   3.7980000972747803 ]
    ur_dc_flat_mat[8] = [ 8,   1,   177.66000366210938,   3.7980000972747803 ]
    ur_dc_flat_mat[9] = [ 9,   1,   171.44000244140625,   3.7980000972747803 ]
    ur_dc_flat_mat[10] = [ 10,   1,   189.21000671386719,  3.7980000972747803 ]
    ur_dc_flat_mat[11] = [ 11,   1,   177.3800048828125,   3.7980000972747803 ]
    ur_dc_tou_mat = [[ 1,   1,   9e+37,   0 ]]
    dc_enable = 0

elif Customer in ["MGS"]:
    install_cost = 1.29*1000*system_size 
    discount_rate = 6.62
    
    ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
    ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    if Scheme == 1:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 2
        NEM_sale_rate = 0
    elif Scheme == 2:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 0
        NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
    elif Scheme == 3:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   buyback_TOU_net_billing_peak_values[Sensitivity] ]
        metering = 2
        NEM_sale_rate = 0
    
    fixed_charge = 8.921
    ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
    ur_dc_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    ur_dc_flat_mat = [None]*12
    ur_dc_flat_mat[0] = [ 0,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[1] = [ 1,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[2] = [ 2,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[3] = [ 3,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[4] = [ 4,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[5] = [ 5,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[6] = [ 6,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[7] = [ 7,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[8] = [ 8,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[9] = [ 9,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[10] = [ 10,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[11] = [ 11,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_tou_mat = [None]*2
    ur_dc_tou_mat[0] = [ 2,   1,   9e+37,   0 ]
    ur_dc_tou_mat[1] = [ 1,   1,   9e+37,   0 ]
    dc_enable = 1

elif Customer in ["LGS", "LGS_BOI"]:
    if Customer == "LGS":
        install_cost = 1.14*1000*system_size 
    elif Customer == "LGS_BOI":
        install_cost = 1.14*1000*system_size*0.9 # *0.9: assuming BOI incentive give taxable deduction of 0.2*0.5*investment cost

    discount_rate = 6.62
    ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
    ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    if Scheme == 1:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 2
        NEM_sale_rate = 0
    elif Scheme == 2:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
        metering = 0
        NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
    elif Scheme == 3:
        ur_ec_tou_mat = [None]*2
        ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
        ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   buyback_TOU_net_billing_peak_values[Sensitivity] ]
        metering = 2
        NEM_sale_rate = 0

    fixed_charge = 8.921
    ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
    ur_dc_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
    ur_dc_flat_mat = [None]*12
    ur_dc_flat_mat[0] = [ 0,   1,  9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[1] = [ 1,   1,  9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[2] = [ 2,   1,  9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[3] = [ 3,   1,  9e+37,   3.7980000972747803 ] 
    ur_dc_flat_mat[4] = [ 4,   1,  9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[5] =[ 5,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[6] =[ 6,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[7] =[ 7,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[8] =[ 8,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[9] =[ 9,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[10] =[ 10,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_flat_mat[11] =[ 11,   1,   9e+37,   3.7980000972747803 ]
    ur_dc_tou_mat = [None]*2
    ur_dc_tou_mat[0] = [ 2,   1,   9e+37,   0 ]
    ur_dc_tou_mat[1] = [ 1,   1,   9e+37,   0 ]
    dc_enable = 1

Now we start SAM Simulation program.

In [24]:
#Simulation
if __name__ == "__main__":
    ssc = PySSC()
    #ssc = sscapi.PySSC()
    print ('Current folder = Solar NEM (ERI)/Data manipulation/SAM')
    print ('SSC Version = ', ssc.version())
    print ('SSC Build Information = ', ssc.build_info().decode("utf - 8"))
    ssc.module_exec_set_print(0)
    data = ssc.data_create()
    ssc.data_set_number( data, b'itc_fed_percent', 0 )
    om_fixed =[ 0 ];
    ssc.data_set_array( data, b'om_fixed',  om_fixed);
    ssc.data_set_number( data, b'pbi_oth_escal', 0 )
    occ_schedule =[ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ];
    ssc.data_set_array( data, b'occ_schedule',  occ_schedule);
    ssc.data_set_number( data, b'cbi_uti_tax_fed', 1 )
    ssc.data_set_number( data, b'cbi_fed_deprbas_fed', 0 )
    ssc.data_set_number( data, b'dc_ac_ratio', 1.1000000238418579 )
    #Input data
    ssc.data_set_string( data, b'solar_resource_file', weather_file);
    #ur_ec_sched_weekday = mat_TOU_weekday;

    ssc.data_set_matrix( data, b'ur_ec_sched_weekday', ur_ec_sched_weekday );
    ssc.data_set_number( data, b'itc_sta_amount_deprbas_sta', 0 )
    ssc.data_set_number( data, b'debt_fraction', 0 )
    ssc.data_set_number( data, b'cbi_sta_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'ibi_uti_percent_tax_sta', 1 )

    #Input data
    #ur_dc_sched_weekend = mat_TOU_weekend;
    ssc.data_set_matrix( data, b'ur_dc_sched_weekend', ur_dc_sched_weekend );

    ssc.data_set_number( data, b'en_fridge', 1 )
    ssc.data_set_number( data, b'itc_fed_amount', 0 )
    pbi_sta_amount =[ 0 ];
    ssc.data_set_array( data, b'pbi_sta_amount',  pbi_sta_amount);
    ssc.data_set_number( data, b'cbi_oth_deprbas_fed', 0 )
    ssc.data_set_number( data, b'cbi_sta_deprbas_fed', 0 )
    ssc.data_set_number( data, b'ibi_oth_percent_maxvalue', 9.9999996802856925e+37 )
    #Input data on TOU rate
    #ur_ec_tou_mat = [[ 1,   1,   30000000,   0,   TOU_buy1,   TOU_sell1 ], [ 2,   1,   30000000,   0,   TOU_buy2,   TOU_sell2 ], [ 3,   0,   0,   0,   0,   0 ]];
    ssc.data_set_matrix( data, b'ur_ec_tou_mat', ur_ec_tou_mat );
    ssc.data_set_number( data, b'loan_term', 0 )
    ssc.data_set_number( data, b'ur_metering_option', metering )
    ssc.data_set_number( data, b'inv_eff', 96 )
    ssc.data_set_number( data, b'ibi_uti_amount_tax_fed', 1 )
    #Input data
    ssc.data_set_number( data, b'ur_monthly_fixed_charge', fixed_charge )
    ssc.data_set_number( data, b'en_wash', 1 )
    ssc.data_set_number( data, b'pbi_fed_tax_fed', 1 )
    ssc.data_set_number( data, b'ibi_sta_amount_deprbas_sta', 0 )
    pbi_oth_amount =[ 0 ];
    ssc.data_set_array( data, b'pbi_oth_amount',  pbi_oth_amount);
    ssc.data_set_number( data, b'pbi_sta_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_fed_percent_deprbas_sta', 0 )
    ssc.data_set_number( data, b'yrbuilt', 1980 )
    ssc.data_set_number( data, b'en_heat', 1 )
    monthly_util =[ 725, 630, 665, 795, 1040, 1590, 1925, 1730, 1380, 1080, 635, 715 ];
    ssc.data_set_array( data, b'monthly_util',  monthly_util);
    ssc.data_set_number( data, b'pbi_uti_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_oth_percent', 0 )
    ssc.data_set_number( data, b'en_dry', 1 )
    ssc.data_set_number( data, b'cbi_sta_deprbas_sta', 0 )
    ssc.data_set_number( data, b'state_tax_rate', 0 )
    ssc.data_set_number( data, b'federal_tax_rate', 0 )
    ssc.data_set_number( data, b'pbi_uti_escal', 0 )
    ssc.data_set_number( data, b'ibi_sta_amount', 0 )
    ssc.data_set_number( data, b'ibi_oth_amount', 0 )
    om_capacity =[ 10 ];
    ssc.data_set_array( data, b'om_capacity',  om_capacity);
    ssc.data_set_number( data, b'cbi_oth_tax_fed', 1 )
    ssc.data_set_number( data, b'floor_area', 2000 )
    ssc.data_set_number( data, b'cbi_uti_deprbas_fed', 0 )
    ssc.data_set_number( data, b'itc_sta_percent_deprbas_sta', 0 )
    #ur_dc_sched_weekday = [[ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ]];
    ssc.data_set_matrix( data, b'ur_dc_sched_weekday', ur_dc_sched_weekday );
    ssc.data_set_number( data, b'ibi_fed_amount_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_uti_percent_tax_fed', 1 )
    ssc.data_set_number( data, b'tcool', 76 )
    ssc.data_set_number( data, b'adjust:constant', 0 )
    ssc.data_set_number( data, b'ibi_oth_amount_deprbas_sta', 0 )
    ssc.data_set_number( data, b'ibi_oth_amount_tax_fed', 1 )
    ssc.data_set_number( data, b'itc_sta_percent_deprbas_fed', 0 )
    ssc.data_set_number( data, b'cbi_uti_amount', 0 )
    ssc.data_set_number( data, b'cbi_oth_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'prop_tax_assessed_decline', 0 )
    om_production =[ 0 ];
    ssc.data_set_array( data, b'om_production',  om_production);
    ssc.data_set_number( data, b'itc_fed_percent_deprbas_sta', 1 )
    ssc.data_set_number( data, b'itc_fed_amount_deprbas_sta', 1 )
    pbi_fed_amount =[ 0 ];
    ssc.data_set_array( data, b'pbi_fed_amount',  pbi_fed_amount);
    ssc.data_set_number( data, b'pbi_sta_tax_fed', 1 )
    ssc.data_set_number( data, b'tilt', 13.699999809265137 )
    load_escalation =[ 3.5 ];
    ssc.data_set_array( data, b'load_escalation',  load_escalation);
    ssc.data_set_number( data, b'salvage_percentage', 0 )
    ssc.data_set_number( data, b'ibi_sta_percent_deprbas_sta', 0 )
    ssc.data_set_number( data, b'itc_fed_percent_deprbas_fed', 1 )
    ssc.data_set_number( data, b'property_tax_rate', 0 )
    ssc.data_set_number( data, b'ibi_sta_amount_deprbas_fed', 0 )
    ssc.data_set_number( data, b'theat', 68 )
    ssc.data_set_number( data, b'cbi_oth_amount', 0 )
    ssc.data_set_number( data, b'ibi_uti_percent_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'azimuth', 180 )
    ssc.data_set_number( data, b'pbi_fed_term', 0 )
    ssc.data_set_number( data, b'ur_annual_min_charge', 0 )
    ssc.data_set_number( data, b'itc_fed_amount_deprbas_fed', 1 )
    ssc.data_set_number( data, b'battery_per_kwh', 600 )
    ssc.data_set_number( data, b'ptc_fed_term', 10 )
    ssc.data_set_number( data, b'pbi_oth_tax_sta', 1 )
    ssc.data_set_number( data, b'pbi_sta_escal', 0 )
    ssc.data_set_number( data, b'ibi_oth_amount_deprbas_fed', 0 )
    ssc.data_set_number( data, b'theatsb', 68 )
    ptc_fed_amount =[ 0 ];
    ssc.data_set_array( data, b'ptc_fed_amount',  ptc_fed_amount);
    ssc.data_set_number( data, b'sales_tax_rate', 0 )
    ssc.data_set_number( data, b'insurance_rate', 0.25 )
    ssc.data_set_number( data, b'ptc_sta_escal', 0 )
    ssc.data_set_number( data, b'ur_monthly_min_charge', 0 )
    ssc.data_set_number( data, b'itc_fed_percent_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'ibi_oth_percent_tax_fed', 1 )
    ssc.data_set_number( data, b'cbi_oth_deprbas_sta', 0 )
    ssc.data_set_number( data, b'en_belpe', 0 )
    ssc.data_set_number( data, b'pbi_oth_term', 0 )
    ssc.data_set_number( data, b'cbi_fed_tax_sta', 1 )
    ssc.data_set_number( data, b'analysis_period', 25 )
    ssc.data_set_number( data, b'om_production_escal', 0 )
    ssc.data_set_number( data, b'ibi_sta_amount_tax_sta', 1 )
    ssc.data_set_number( data, b'cbi_sta_tax_fed', 1 )
    ssc.data_set_number( data, b'om_capacity_escal', 2.2000000476837158 )
    ssc.data_set_number( data, b'ibi_fed_amount_deprbas_sta', 0 )
    ssc.data_set_number( data, b'loan_rate', 0 )
    ssc.data_set_number( data, b'pbi_oth_tax_fed', 1 )
    ssc.data_set_number( data, b'ibi_fed_percent_deprbas_fed', 0 )
    #ur_dc_flat_mat = [[ 0,   1,   168.69999694824219,   3.7980000972747803 ], [ 1,   1,   200,   3.7980000972747803 ], [ 2,   1,   178.60000610351562,   3.7980000972747803 ], [ 3,   1,   193.1300048828125,   3.7980000972747803 ], [ 4,   1,   189.92999267578125,   3.7980000972747803 ], [ 5,   1,   184.92999267578125,   3.7980000972747803 ], [ 6,   1,   176.6199951171875,   3.7980000972747803 ], [ 7,   1,   168.80000305175781,   3.7980000972747803 ], [ 8,   1,   177.66000366210938,   3.7980000972747803 ], [ 9,   1,   171.44000244140625,   3.7980000972747803 ], [ 10,   1,   189.21000671386719,   3.7980000972747803 ], [ 11,   1,   177.3800048828125,   3.7980000972747803 ]];
    ssc.data_set_matrix( data, b'ur_dc_flat_mat', ur_dc_flat_mat );
    ssc.data_set_number( data, b'cbi_fed_tax_fed', 1 )
    ssc.data_set_number( data, b'ibi_sta_percent_deprbas_fed', 0 )
    ssc.data_set_number( data, b'cbi_sta_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_fed_amount_deprbas_fed', 0 )
    ssc.data_set_number( data, b'system_use_lifetime_output', 0 )
    ssc.data_set_number( data, b'en_mels', 1 )
    t_sched =[ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ];
    ssc.data_set_array( data, b't_sched',  t_sched);
    ssc.data_set_number( data, b'mortgage', 0 )
    ssc.data_set_number( data, b'pbi_fed_tax_sta', 1 )
    ssc.data_set_number( data, b'array_type', 0 )
    ssc.data_set_number( data, b'pbi_uti_term', 0 )
    ssc.data_set_array_from_csv( data, b'load', load_file);
    ssc.data_set_number( data, b'ibi_fed_percent_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'ibi_fed_amount_tax_fed', 1 )
    ssc.data_set_number( data, b'total_installed_cost', install_cost )
    ssc.data_set_number( data, b'cbi_uti_deprbas_sta', 0 )
    ssc.data_set_number( data, b'ptc_sta_term', 10 )
    degradation =[ 0.5 ];
    ssc.data_set_array( data, b'degradation',  degradation);
    ssc.data_set_number( data, b'itc_sta_amount_deprbas_fed', 0 )
    ssc.data_set_number( data, b'en_dish', 1 )
    ssc.data_set_number( data, b'ur_nm_yearend_sell_rate', NEM_sale_rate )
    ssc.data_set_number( data, b'ptc_fed_escal', 0 )
    ssc.data_set_number( data, b'ibi_sta_percent_tax_fed', 1 )
    ssc.data_set_number( data, b'cbi_fed_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'ibi_oth_amount_tax_sta', 1 )
    ssc.data_set_number( data, b'itc_sta_amount', 0 )
    ssc.data_set_number( data, b'ibi_fed_percent_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_sta_amount_tax_fed', 1 )
    #ur_dc_tou_mat = [[ 1,   1,   9.9999996802856925e+37,   0 ]];
    ssc.data_set_matrix( data, b'ur_dc_tou_mat', ur_dc_tou_mat );
    ssc.data_set_number( data, b'ibi_uti_amount_deprbas_fed', 0 )
    ssc.data_set_number( data, b'ibi_sta_percent_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'om_fixed_escal', 0 )
    ssc.data_set_number( data, b'ibi_uti_amount_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_sta_percent_tax_sta', 1 )
    ssc.data_set_number( data, b'cbi_sta_amount', 0 )
    ssc.data_set_number( data, b'occupants', 4 )
    #Input data for system capacity
    ssc.data_set_number( data, b'system_capacity', system_size )
    ssc.data_set_number( data, b'om_fuel_cost_escal', 0 )
    ssc.data_set_number( data, b'pbi_uti_tax_fed', 1 )
    ssc.data_set_number( data, b'ibi_oth_percent_deprbas_fed', 0 )
    ssc.data_set_number( data, b'ibi_uti_amount_deprbas_sta', 0 )
    rate_escalation =[ 3.5 ];
    ssc.data_set_array( data, b'rate_escalation',  rate_escalation);
    pbi_uti_amount =[ 0 ];
    ssc.data_set_array( data, b'pbi_uti_amount',  pbi_uti_amount);
    ssc.data_set_number( data, b'ibi_fed_percent_tax_fed', 1 )
    ssc.data_set_number( data, b'cbi_uti_tax_sta', 1 )
    ssc.data_set_number( data, b'pbi_fed_escal', 0 )
    #ur_ec_sched_weekend = [[ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ]];
    ssc.data_set_matrix( data, b'ur_ec_sched_weekend', ur_ec_sched_weekend );
    ssc.data_set_number( data, b'prop_tax_cost_assessed_percent', 0 )
    ssc.data_set_number( data, b'ur_dc_enable', dc_enable )
    ssc.data_set_number( data, b'pbi_sta_term', 0 )
    ssc.data_set_number( data, b'itc_sta_percent', 0 )
    ssc.data_set_number( data, b'losses', 14.075660705566406 )
    om_fuel_cost =[ 0 ];
    ssc.data_set_array( data, b'om_fuel_cost',  om_fuel_cost);
    ssc.data_set_number( data, b'cbi_uti_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'tcoolsb', 76 )
    ptc_sta_amount =[ 0 ];
    ssc.data_set_array( data, b'ptc_sta_amount',  ptc_sta_amount);
    ssc.data_set_number( data, b'ibi_uti_percent_deprbas_sta', 0 )
    ssc.data_set_number( data, b'inflation_rate', 2.5 )
    ssc.data_set_number( data, b'ibi_fed_amount', 0 )
    ssc.data_set_number( data, b'ibi_uti_percent_deprbas_fed', 0 )
    ssc.data_set_number( data, b'real_discount_rate', discount_rate )
    ssc.data_set_number( data, b'module_type', 0 )
    ssc.data_set_number( data, b'market', 0 )
    ssc.data_set_number( data, b'ibi_uti_percent', 0 )
    ssc.data_set_number( data, b'stories', 2 )
    ssc.data_set_number( data, b'ibi_oth_percent_deprbas_sta', 0 )
    ssc.data_set_number( data, b'ibi_fed_percent', 0 )
    ssc.data_set_number( data, b'en_cool', 1 )
    ssc.data_set_number( data, b'cbi_fed_amount', 0 )
    ssc.data_set_number( data, b'ibi_uti_amount', 0 )
    ssc.data_set_number( data, b'retrofits', 0 )
    ssc.data_set_number( data, b'en_range', 1 )
    ssc.data_set_number( data, b'gcr', 0.40000000596046448 )
    ssc.data_set_number( data, b'cbi_fed_deprbas_sta', 0 )
    ssc.data_set_number( data, b'itc_sta_percent_maxvalue', 9.9999996802856925e+37 )
    ssc.data_set_number( data, b'ibi_oth_percent_tax_sta', 1 )
    ssc.data_set_number( data, b'ibi_sta_percent', 0 )
    ssc.data_set_number( data, b'cbi_oth_tax_sta', 1 )
    module = ssc.module_create(b'pvwattsv5')	
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('pvwattsv5 simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    module = ssc.module_create(b'belpe')	
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('belpe simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    module = ssc.module_create(b'utilityrate4')	
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('utilityrate4 simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    module = ssc.module_create(b'cashloan')	
    ssc.module_exec_set_print( 0 );
    if ssc.module_exec(module, data) == 0:
        print ('cashloan simulation error')
        idx = 1
        msg = ssc.module_log(module, 0)
        while (msg != None):
            print ('	: ' + msg.decode("utf - 8"))
            msg = ssc.module_log(module, idx)
            idx = idx + 1
        SystemExit( "Simulation Error" );
    ssc.module_free(module)
    annual_energy = ssc.data_get_number(data, b'annual_energy');
    #print ('Annual energy (year 1) = ', annual_energy)
    capacity_factor = ssc.data_get_number(data, b'capacity_factor');
    #print ('Capacity factor (year 1) = ', capacity_factor)
    kwh_per_kw = ssc.data_get_number(data, b'kwh_per_kw');
    #print ('Energy yield (year 1) = ', kwh_per_kw)
    lcoe_nom = ssc.data_get_number(data, b'lcoe_nom');
    #print ('Levelized COE (nominal) = ', lcoe_nom)
    lcoe_real = ssc.data_get_number(data, b'lcoe_real');
    #print ('Levelized COE (real) = ', lcoe_real)
    elec_cost_without_system_year1 = ssc.data_get_number(data, b'elec_cost_without_system_year1');
    #print ('Electricity bill without system (year 1) = ', elec_cost_without_system_year1)
    elec_cost_with_system_year1 = ssc.data_get_number(data, b'elec_cost_with_system_year1');
    #print ('Electricity bill with system (year 1) = ', elec_cost_with_system_year1)
    savings_year1 = ssc.data_get_number(data, b'savings_year1');
    #print ('Net savings with system (year 1) = ', savings_year1)
    npv = ssc.data_get_number(data, b'npv');
    #print ('Net present value = ', npv)
    payback = ssc.data_get_number(data, b'payback');
    #print ('Payback period = ', payback)
    adjusted_installed_cost = ssc.data_get_number(data, b'adjusted_installed_cost');
    #print ('Net capital cost = ', adjusted_installed_cost)
    first_cost = ssc.data_get_number(data, b'first_cost');
    #print ('Equity = ', first_cost)
    loan_amount = ssc.data_get_number(data, b'loan_amount');
    #print ('Debt = ', loan_amount)
    cash_flow = ssc.data_get_array(data,b'cf_after_tax_cash_flow');
    #print ('Cash flow = ', cash_flow)

    #calculate IRR using NumPy
    IRR = numpy.irr(cash_flow)

    hourly_energy = ssc.data_get_array(data,b'gen');
    hourly_load = ssc.data_get_array(data, b'load');
    hourly_to_grid = ssc.data_get_array(data, b'year1_hourly_e_togrid');
    hourly_from_grid = ssc.data_get_array(data, b'year1_hourly_e_fromgrid');
    hourly_system_to_load = ssc.data_get_array(data, b'year1_hourly_system_to_load');


Current folder = Solar NEM (ERI)/Data manipulation/SAM
('SSC Version = ', 160)
('SSC Build Information = ', u'Windows 64 bit Visual C++ Sep 22 2016 03:34:19')


Produce the list of outputs we need. Skip the second result on detailed hourly results for now.

In [25]:
result_1 = [Customer, \
Scheme, \
Sensitivity, \
system_size, \
NEM_sale_rate,\
Region, \
annual_energy, \
capacity_factor, \
kwh_per_kw, \
lcoe_nom, \
lcoe_real, \
elec_cost_without_system_year1, \
elec_cost_with_system_year1, \
savings_year1, \
npv, \
payback, \
IRR, \
adjusted_installed_cost, \
weather_file]

#write detailed, hourly results
# if Sensitivity == 2:
# 	month = zip([Customer]*8760, \
# 	[Scheme]*8760, \
# 	[Sensitivity]*8760, \
# 	[Region]*8760, \
# 	[1]*24*31 + [2]*24*28 + [3]*24*31 + [4]*24*30 + [5]*24*31 + [6]*24*30 + [7]*24*31 + [8]*24*31 + [9]*24*30 + [10]*24*31 + [11]*24*30 + [12]*24*31, \
# 	[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]*365, \
# 	hourly_energy, \
# 	hourly_load, \
# 	hourly_to_grid, \
# 	hourly_from_grid, \
# 	hourly_system_to_load)

# 	for row in month:
# 		result_file2.writerow(row)

# ssc.data_free(data);
# return;

In [27]:
print(result_1)

['Res', 1, 1, 0, 16, 4068.09228515625, 15.479803085327148, 1356.03076171875, 12.564873695373535, 9.62972640991211, 763.0892333984375, 484.712890625, 278.3763427734375, 2562.30712890625, 13.478580474853516, 0.08089117661056111, 5790.0, 'C:/Users/Admin/Dropbox/Solar NEM (ERI)/Data manipulation/SAM/THA_weather_data/THA_ARANYAPRATHET_484620_IW2/THA_ARANYAPRATHET_484620_IW2.EPW']


Now, we are ready to pack them back as a function.

In [30]:
def simulate(Customer, Scheme, Sensitivity, weather_file, Region, system_size):
    ###############################################################################################################
    #Basic buyback rate
    if Customer in ["Res","SGS"] :
        block1 = 0.083
        block2 = 0.111
        block3 = 0.117
        block_average = (block1 + block2 + block3)/3 #Not using it after all?

    if Customer in ["Res_TOU","SGS_TOU"] :
        TOU_retail_1 = 0.066
        TOU_retail_2 = 0.156
        TOU_retail_average = (TOU_retail_1 + TOU_retail_2)/2

        TOU_wholesale_1 = 0.057823
        TOU_wholesale_2 = 0.111183
        TOU_wholesale_average = (TOU_wholesale_1 + TOU_wholesale_2)/2

    elif Customer in ["MGS","LGS", "LGS_BOI"]:
        TOU_retail_1 = 0.066
        TOU_retail_2 = 0.111
        TOU_retail_average = (TOU_retail_1 + TOU_retail_2)/2

        TOU_wholesale_1 = 0.057106
        TOU_wholesale_2 = 0.087409
        TOU_wholesale_average = (TOU_wholesale_1 + TOU_wholesale_2)/2
    #################################################################################################################
    #Buyback rate for different customers and sensitivies (Sensitivity: 1 = below retail, 2 = retail, 3 = above retail)
    if Customer in ["Res","SGS"] :
        buyback_block_values = [0,                  1/35,               2/35,                    3/35]
    if Customer in ["Res_TOU","SGS_TOU","MGS","LGS", "LGS_BOI"]:
        buyback_TOU_net_metering_values = [0, TOU_wholesale_average, TOU_retail_average, TOU_retail_average*1.05]
        buyback_TOU_net_billing_offpeak_values = [0,               1/35,               2/35,               3/35] 
        buyback_TOU_net_billing_peak_values = [0,                1/35,               2/35,                 3/35]
    ##################################################################################################################
    #Set load profile based on Customer Type
    if Customer in ["Res", "Res_TOU"]:
        Customer_type = b'11'
    elif Customer in ["SGS", "SGS_TOU"]:
        Customer_type = b'20'
    elif Customer == "MGS":
        Customer_type = b'30'
    elif Customer in ["LGS", "LGS_BOI"]:
        Customer_type = b'40' 

    load_file = DataDir + '/2015/load profile/DATA' + str(Region).encode('utf-8') + '_' + Customer_type + '.csv'
    #################################################################################################################
    #Fill in necessary values
    if Customer in ["Res","SGS"] :
        install_cost = 1.93*1000*system_size 
        discount_rate = 2.67

        ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12

        if Scheme == 1: #pilot
            ur_ec_tou_mat = [None]*3
            ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   0 ]
            ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   0 ]
            ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   0 ]
            metering = 2
            NEM_sale_rate = 0

        elif Scheme == 2: #net-metering
            ur_ec_tou_mat = [None]*3
            ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   0 ]
            ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   0 ]
            ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   0 ]
            metering = 0
            NEM_sale_rate = buyback_block_values[Sensitivity]

        elif Scheme == 3: #net-billing
            ur_ec_tou_mat = [None]*3
            ur_ec_tou_mat[0] = [ 1,   1,   150,         0,   block1,   buyback_block_values[Sensitivity] ]
            ur_ec_tou_mat[1] = [ 1,   2,   400,         0,   block2,   buyback_block_values[Sensitivity] ]
            ur_ec_tou_mat[2] = [ 1,   3,   300000000,   0,   block3,   buyback_block_values[Sensitivity] ]
            metering = 2
            NEM_sale_rate = 0

        fixed_charge = 1.0920000076293945
        ur_dc_sched_weekday = ur_ec_sched_weekday
        ur_dc_sched_weekend = ur_dc_sched_weekday
        ur_dc_flat_mat = [None]*12
        ur_dc_flat_mat[0] = [ 0,   1,   168.69999694824219,   3.7980000972747803 ]
        ur_dc_flat_mat[1] = [ 1,   1,   200,                  3.7980000972747803 ]
        ur_dc_flat_mat[2] = [ 2,   1,   178.60000610351562,   3.7980000972747803 ]
        ur_dc_flat_mat[3] = [ 3,   1,   193.1300048828125,    3.7980000972747803 ]
        ur_dc_flat_mat[4] = [ 4,   1,   189.92999267578125,   3.7980000972747803 ]
        ur_dc_flat_mat[5] = [ 5,   1,   184.92999267578125,   3.7980000972747803 ]
        ur_dc_flat_mat[6] = [ 6,   1,   176.6199951171875,    3.7980000972747803 ]
        ur_dc_flat_mat[7] = [ 7,   1,   168.80000305175781,   3.7980000972747803 ]
        ur_dc_flat_mat[8] = [ 8,   1,   177.66000366210938,   3.7980000972747803 ]
        ur_dc_flat_mat[9] = [ 9,   1,   171.44000244140625,   3.7980000972747803 ]
        ur_dc_flat_mat[10] = [ 10,   1,   189.21000671386719,  3.7980000972747803 ]
        ur_dc_flat_mat[11] = [ 11,   1,   177.3800048828125,   3.7980000972747803 ]
        ur_dc_tou_mat = [[ 1,   1,   9e+37,   0 ]]
        dc_enable = 0

    elif Customer in ["Res_TOU","SGS_TOU"] :
        install_cost = 1.93*1000*system_size 
        discount_rate = 2.67
        ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
        ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        if Scheme == 1:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 2
            NEM_sale_rate = 0
        elif Scheme == 2:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 0
            NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
        elif Scheme == 3:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,  buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,  buyback_TOU_net_billing_peak_values[Sensitivity] ]
            metering = 2
            NEM_sale_rate = 0

        fixed_charge = 1.0920000076293945
        ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        ur_dc_sched_weekend = ur_dc_sched_weekday 
        ur_dc_flat_mat = [None]*12
        ur_dc_flat_mat[0] = [ 0,   1,   168.69999694824219,   3.7980000972747803 ]
        ur_dc_flat_mat[1] = [ 1,   1,   200,                  3.7980000972747803 ]
        ur_dc_flat_mat[2] = [ 2,   1,   178.60000610351562,   3.7980000972747803 ]
        ur_dc_flat_mat[3] = [ 3,   1,   193.1300048828125,    3.7980000972747803 ]
        ur_dc_flat_mat[4] = [ 4,   1,   189.92999267578125,   3.7980000972747803 ]
        ur_dc_flat_mat[5] = [ 5,   1,   184.92999267578125,   3.7980000972747803 ]
        ur_dc_flat_mat[6] = [ 6,   1,   176.6199951171875,    3.7980000972747803 ] 
        ur_dc_flat_mat[7] = [ 7,   1,   168.80000305175781,   3.7980000972747803 ]
        ur_dc_flat_mat[8] = [ 8,   1,   177.66000366210938,   3.7980000972747803 ]
        ur_dc_flat_mat[9] = [ 9,   1,   171.44000244140625,   3.7980000972747803 ]
        ur_dc_flat_mat[10] = [ 10,   1,   189.21000671386719,  3.7980000972747803 ]
        ur_dc_flat_mat[11] = [ 11,   1,   177.3800048828125,   3.7980000972747803 ]
        ur_dc_tou_mat = [[ 1,   1,   9e+37,   0 ]]
        dc_enable = 0

    elif Customer in ["MGS"]:
        install_cost = 1.29*1000*system_size 
        discount_rate = 6.62

        ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
        ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        if Scheme == 1:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 2
            NEM_sale_rate = 0
        elif Scheme == 2:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 0
            NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
        elif Scheme == 3:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   buyback_TOU_net_billing_peak_values[Sensitivity] ]
            metering = 2
            NEM_sale_rate = 0

        fixed_charge = 8.921
        ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
        ur_dc_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        ur_dc_flat_mat = [None]*12
        ur_dc_flat_mat[0] = [ 0,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[1] = [ 1,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[2] = [ 2,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[3] = [ 3,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[4] = [ 4,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[5] = [ 5,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[6] = [ 6,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[7] = [ 7,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[8] = [ 8,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[9] = [ 9,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[10] = [ 10,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[11] = [ 11,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_tou_mat = [None]*2
        ur_dc_tou_mat[0] = [ 2,   1,   9e+37,   0 ]
        ur_dc_tou_mat[1] = [ 1,   1,   9e+37,   0 ]
        dc_enable = 1

    elif Customer in ["LGS", "LGS_BOI"]:
        if Customer == "LGS":
            install_cost = 1.14*1000*system_size 
        elif Customer == "LGS_BOI":
            install_cost = 1.14*1000*system_size*0.9 # *0.9: assuming BOI incentive give taxable deduction of 0.2*0.5*investment cost

        discount_rate = 6.62
        ur_ec_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
        ur_ec_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        if Scheme == 1:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 2
            NEM_sale_rate = 0
        elif Scheme == 2:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   0 ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   0 ]
            metering = 0
            NEM_sale_rate = buyback_TOU_net_metering_values[Sensitivity]
        elif Scheme == 3:
            ur_ec_tou_mat = [None]*2
            ur_ec_tou_mat[0] = [ 1,   1,   9e+37,   0,   TOU_retail_1,   buyback_TOU_net_billing_offpeak_values[Sensitivity] ]
            ur_ec_tou_mat[1] = [ 2,   1,   9e+37,   0,   TOU_retail_2,   buyback_TOU_net_billing_peak_values[Sensitivity] ]
            metering = 2
            NEM_sale_rate = 0

        fixed_charge = 8.921
        ur_dc_sched_weekday = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]]*12
        ur_dc_sched_weekend = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]*12
        ur_dc_flat_mat = [None]*12
        ur_dc_flat_mat[0] = [ 0,   1,  9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[1] = [ 1,   1,  9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[2] = [ 2,   1,  9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[3] = [ 3,   1,  9e+37,   3.7980000972747803 ] 
        ur_dc_flat_mat[4] = [ 4,   1,  9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[5] =[ 5,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[6] =[ 6,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[7] =[ 7,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[8] =[ 8,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[9] =[ 9,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[10] =[ 10,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_flat_mat[11] =[ 11,   1,   9e+37,   3.7980000972747803 ]
        ur_dc_tou_mat = [None]*2
        ur_dc_tou_mat[0] = [ 2,   1,   9e+37,   0 ]
        ur_dc_tou_mat[1] = [ 1,   1,   9e+37,   0 ]
        dc_enable = 1
    ################################################################################################################
    #Simulation
    if __name__ == "__main__":
        ssc = PySSC()
        ssc.module_exec_set_print(0)
        data = ssc.data_create()
        ssc.data_set_number( data, b'itc_fed_percent', 0 )
        om_fixed =[ 0 ];
        ssc.data_set_array( data, b'om_fixed',  om_fixed);
        ssc.data_set_number( data, b'pbi_oth_escal', 0 )
        occ_schedule =[ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ];
        ssc.data_set_array( data, b'occ_schedule',  occ_schedule);
        ssc.data_set_number( data, b'cbi_uti_tax_fed', 1 )
        ssc.data_set_number( data, b'cbi_fed_deprbas_fed', 0 )
        ssc.data_set_number( data, b'dc_ac_ratio', 1.1000000238418579 )
        #Input data
        ssc.data_set_string( data, b'solar_resource_file', weather_file);
        #ur_ec_sched_weekday = mat_TOU_weekday;

        ssc.data_set_matrix( data, b'ur_ec_sched_weekday', ur_ec_sched_weekday );
        ssc.data_set_number( data, b'itc_sta_amount_deprbas_sta', 0 )
        ssc.data_set_number( data, b'debt_fraction', 0 )
        ssc.data_set_number( data, b'cbi_sta_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'ibi_uti_percent_tax_sta', 1 )

        #Input data
        #ur_dc_sched_weekend = mat_TOU_weekend;
        ssc.data_set_matrix( data, b'ur_dc_sched_weekend', ur_dc_sched_weekend );

        ssc.data_set_number( data, b'en_fridge', 1 )
        ssc.data_set_number( data, b'itc_fed_amount', 0 )
        pbi_sta_amount =[ 0 ];
        ssc.data_set_array( data, b'pbi_sta_amount',  pbi_sta_amount);
        ssc.data_set_number( data, b'cbi_oth_deprbas_fed', 0 )
        ssc.data_set_number( data, b'cbi_sta_deprbas_fed', 0 )
        ssc.data_set_number( data, b'ibi_oth_percent_maxvalue', 9.9999996802856925e+37 )
        #Input data on TOU rate
        #ur_ec_tou_mat = [[ 1,   1,   30000000,   0,   TOU_buy1,   TOU_sell1 ], [ 2,   1,   30000000,   0,   TOU_buy2,   TOU_sell2 ], [ 3,   0,   0,   0,   0,   0 ]];
        ssc.data_set_matrix( data, b'ur_ec_tou_mat', ur_ec_tou_mat );
        ssc.data_set_number( data, b'loan_term', 0 )
        ssc.data_set_number( data, b'ur_metering_option', metering )
        ssc.data_set_number( data, b'inv_eff', 96 )
        ssc.data_set_number( data, b'ibi_uti_amount_tax_fed', 1 )
        #Input data
        ssc.data_set_number( data, b'ur_monthly_fixed_charge', fixed_charge )
        ssc.data_set_number( data, b'en_wash', 1 )
        ssc.data_set_number( data, b'pbi_fed_tax_fed', 1 )
        ssc.data_set_number( data, b'ibi_sta_amount_deprbas_sta', 0 )
        pbi_oth_amount =[ 0 ];
        ssc.data_set_array( data, b'pbi_oth_amount',  pbi_oth_amount);
        ssc.data_set_number( data, b'pbi_sta_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_fed_percent_deprbas_sta', 0 )
        ssc.data_set_number( data, b'yrbuilt', 1980 )
        ssc.data_set_number( data, b'en_heat', 1 )
        monthly_util =[ 725, 630, 665, 795, 1040, 1590, 1925, 1730, 1380, 1080, 635, 715 ];
        ssc.data_set_array( data, b'monthly_util',  monthly_util);
        ssc.data_set_number( data, b'pbi_uti_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_oth_percent', 0 )
        ssc.data_set_number( data, b'en_dry', 1 )
        ssc.data_set_number( data, b'cbi_sta_deprbas_sta', 0 )
        ssc.data_set_number( data, b'state_tax_rate', 0 )
        ssc.data_set_number( data, b'federal_tax_rate', 0 )
        ssc.data_set_number( data, b'pbi_uti_escal', 0 )
        ssc.data_set_number( data, b'ibi_sta_amount', 0 )
        ssc.data_set_number( data, b'ibi_oth_amount', 0 )
        om_capacity =[ 10 ];
        ssc.data_set_array( data, b'om_capacity',  om_capacity);
        ssc.data_set_number( data, b'cbi_oth_tax_fed', 1 )
        ssc.data_set_number( data, b'floor_area', 2000 )
        ssc.data_set_number( data, b'cbi_uti_deprbas_fed', 0 )
        ssc.data_set_number( data, b'itc_sta_percent_deprbas_sta', 0 )
        #ur_dc_sched_weekday = [[ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ], [ 1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1 ]];
        ssc.data_set_matrix( data, b'ur_dc_sched_weekday', ur_dc_sched_weekday );
        ssc.data_set_number( data, b'ibi_fed_amount_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_uti_percent_tax_fed', 1 )
        ssc.data_set_number( data, b'tcool', 76 )
        ssc.data_set_number( data, b'adjust:constant', 0 )
        ssc.data_set_number( data, b'ibi_oth_amount_deprbas_sta', 0 )
        ssc.data_set_number( data, b'ibi_oth_amount_tax_fed', 1 )
        ssc.data_set_number( data, b'itc_sta_percent_deprbas_fed', 0 )
        ssc.data_set_number( data, b'cbi_uti_amount', 0 )
        ssc.data_set_number( data, b'cbi_oth_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'prop_tax_assessed_decline', 0 )
        om_production =[ 0 ];
        ssc.data_set_array( data, b'om_production',  om_production);
        ssc.data_set_number( data, b'itc_fed_percent_deprbas_sta', 1 )
        ssc.data_set_number( data, b'itc_fed_amount_deprbas_sta', 1 )
        pbi_fed_amount =[ 0 ];
        ssc.data_set_array( data, b'pbi_fed_amount',  pbi_fed_amount);
        ssc.data_set_number( data, b'pbi_sta_tax_fed', 1 )
        ssc.data_set_number( data, b'tilt', 13.699999809265137 )
        load_escalation =[ 3.5 ];
        ssc.data_set_array( data, b'load_escalation',  load_escalation);
        ssc.data_set_number( data, b'salvage_percentage', 0 )
        ssc.data_set_number( data, b'ibi_sta_percent_deprbas_sta', 0 )
        ssc.data_set_number( data, b'itc_fed_percent_deprbas_fed', 1 )
        ssc.data_set_number( data, b'property_tax_rate', 0 )
        ssc.data_set_number( data, b'ibi_sta_amount_deprbas_fed', 0 )
        ssc.data_set_number( data, b'theat', 68 )
        ssc.data_set_number( data, b'cbi_oth_amount', 0 )
        ssc.data_set_number( data, b'ibi_uti_percent_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'azimuth', 180 )
        ssc.data_set_number( data, b'pbi_fed_term', 0 )
        ssc.data_set_number( data, b'ur_annual_min_charge', 0 )
        ssc.data_set_number( data, b'itc_fed_amount_deprbas_fed', 1 )
        ssc.data_set_number( data, b'battery_per_kwh', 600 )
        ssc.data_set_number( data, b'ptc_fed_term', 10 )
        ssc.data_set_number( data, b'pbi_oth_tax_sta', 1 )
        ssc.data_set_number( data, b'pbi_sta_escal', 0 )
        ssc.data_set_number( data, b'ibi_oth_amount_deprbas_fed', 0 )
        ssc.data_set_number( data, b'theatsb', 68 )
        ptc_fed_amount =[ 0 ];
        ssc.data_set_array( data, b'ptc_fed_amount',  ptc_fed_amount);
        ssc.data_set_number( data, b'sales_tax_rate', 0 )
        ssc.data_set_number( data, b'insurance_rate', 0.25 )
        ssc.data_set_number( data, b'ptc_sta_escal', 0 )
        ssc.data_set_number( data, b'ur_monthly_min_charge', 0 )
        ssc.data_set_number( data, b'itc_fed_percent_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'ibi_oth_percent_tax_fed', 1 )
        ssc.data_set_number( data, b'cbi_oth_deprbas_sta', 0 )
        ssc.data_set_number( data, b'en_belpe', 0 )
        ssc.data_set_number( data, b'pbi_oth_term', 0 )
        ssc.data_set_number( data, b'cbi_fed_tax_sta', 1 )
        ssc.data_set_number( data, b'analysis_period', 25 )
        ssc.data_set_number( data, b'om_production_escal', 0 )
        ssc.data_set_number( data, b'ibi_sta_amount_tax_sta', 1 )
        ssc.data_set_number( data, b'cbi_sta_tax_fed', 1 )
        ssc.data_set_number( data, b'om_capacity_escal', 2.2000000476837158 )
        ssc.data_set_number( data, b'ibi_fed_amount_deprbas_sta', 0 )
        ssc.data_set_number( data, b'loan_rate', 0 )
        ssc.data_set_number( data, b'pbi_oth_tax_fed', 1 )
        ssc.data_set_number( data, b'ibi_fed_percent_deprbas_fed', 0 )
        #ur_dc_flat_mat = [[ 0,   1,   168.69999694824219,   3.7980000972747803 ], [ 1,   1,   200,   3.7980000972747803 ], [ 2,   1,   178.60000610351562,   3.7980000972747803 ], [ 3,   1,   193.1300048828125,   3.7980000972747803 ], [ 4,   1,   189.92999267578125,   3.7980000972747803 ], [ 5,   1,   184.92999267578125,   3.7980000972747803 ], [ 6,   1,   176.6199951171875,   3.7980000972747803 ], [ 7,   1,   168.80000305175781,   3.7980000972747803 ], [ 8,   1,   177.66000366210938,   3.7980000972747803 ], [ 9,   1,   171.44000244140625,   3.7980000972747803 ], [ 10,   1,   189.21000671386719,   3.7980000972747803 ], [ 11,   1,   177.3800048828125,   3.7980000972747803 ]];
        ssc.data_set_matrix( data, b'ur_dc_flat_mat', ur_dc_flat_mat );
        ssc.data_set_number( data, b'cbi_fed_tax_fed', 1 )
        ssc.data_set_number( data, b'ibi_sta_percent_deprbas_fed', 0 )
        ssc.data_set_number( data, b'cbi_sta_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_fed_amount_deprbas_fed', 0 )
        ssc.data_set_number( data, b'system_use_lifetime_output', 0 )
        ssc.data_set_number( data, b'en_mels', 1 )
        t_sched =[ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ];
        ssc.data_set_array( data, b't_sched',  t_sched);
        ssc.data_set_number( data, b'mortgage', 0 )
        ssc.data_set_number( data, b'pbi_fed_tax_sta', 1 )
        ssc.data_set_number( data, b'array_type', 0 )
        ssc.data_set_number( data, b'pbi_uti_term', 0 )
        ssc.data_set_array_from_csv( data, b'load', load_file);
        ssc.data_set_number( data, b'ibi_fed_percent_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'ibi_fed_amount_tax_fed', 1 )
        ssc.data_set_number( data, b'total_installed_cost', install_cost )
        ssc.data_set_number( data, b'cbi_uti_deprbas_sta', 0 )
        ssc.data_set_number( data, b'ptc_sta_term', 10 )
        degradation =[ 0.5 ];
        ssc.data_set_array( data, b'degradation',  degradation);
        ssc.data_set_number( data, b'itc_sta_amount_deprbas_fed', 0 )
        ssc.data_set_number( data, b'en_dish', 1 )
        ssc.data_set_number( data, b'ur_nm_yearend_sell_rate', NEM_sale_rate )
        ssc.data_set_number( data, b'ptc_fed_escal', 0 )
        ssc.data_set_number( data, b'ibi_sta_percent_tax_fed', 1 )
        ssc.data_set_number( data, b'cbi_fed_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'ibi_oth_amount_tax_sta', 1 )
        ssc.data_set_number( data, b'itc_sta_amount', 0 )
        ssc.data_set_number( data, b'ibi_fed_percent_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_sta_amount_tax_fed', 1 )
        #ur_dc_tou_mat = [[ 1,   1,   9.9999996802856925e+37,   0 ]];
        ssc.data_set_matrix( data, b'ur_dc_tou_mat', ur_dc_tou_mat );
        ssc.data_set_number( data, b'ibi_uti_amount_deprbas_fed', 0 )
        ssc.data_set_number( data, b'ibi_sta_percent_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'om_fixed_escal', 0 )
        ssc.data_set_number( data, b'ibi_uti_amount_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_sta_percent_tax_sta', 1 )
        ssc.data_set_number( data, b'cbi_sta_amount', 0 )
        ssc.data_set_number( data, b'occupants', 4 )
        #Input data for system capacity
        ssc.data_set_number( data, b'system_capacity', system_size )
        ssc.data_set_number( data, b'om_fuel_cost_escal', 0 )
        ssc.data_set_number( data, b'pbi_uti_tax_fed', 1 )
        ssc.data_set_number( data, b'ibi_oth_percent_deprbas_fed', 0 )
        ssc.data_set_number( data, b'ibi_uti_amount_deprbas_sta', 0 )
        rate_escalation =[ 3.5 ];
        ssc.data_set_array( data, b'rate_escalation',  rate_escalation);
        pbi_uti_amount =[ 0 ];
        ssc.data_set_array( data, b'pbi_uti_amount',  pbi_uti_amount);
        ssc.data_set_number( data, b'ibi_fed_percent_tax_fed', 1 )
        ssc.data_set_number( data, b'cbi_uti_tax_sta', 1 )
        ssc.data_set_number( data, b'pbi_fed_escal', 0 )
        #ur_ec_sched_weekend = [[ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ], [ 2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2 ]];
        ssc.data_set_matrix( data, b'ur_ec_sched_weekend', ur_ec_sched_weekend );
        ssc.data_set_number( data, b'prop_tax_cost_assessed_percent', 0 )
        ssc.data_set_number( data, b'ur_dc_enable', dc_enable )
        ssc.data_set_number( data, b'pbi_sta_term', 0 )
        ssc.data_set_number( data, b'itc_sta_percent', 0 )
        ssc.data_set_number( data, b'losses', 14.075660705566406 )
        om_fuel_cost =[ 0 ];
        ssc.data_set_array( data, b'om_fuel_cost',  om_fuel_cost);
        ssc.data_set_number( data, b'cbi_uti_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'tcoolsb', 76 )
        ptc_sta_amount =[ 0 ];
        ssc.data_set_array( data, b'ptc_sta_amount',  ptc_sta_amount);
        ssc.data_set_number( data, b'ibi_uti_percent_deprbas_sta', 0 )
        ssc.data_set_number( data, b'inflation_rate', 2.5 )
        ssc.data_set_number( data, b'ibi_fed_amount', 0 )
        ssc.data_set_number( data, b'ibi_uti_percent_deprbas_fed', 0 )
        ssc.data_set_number( data, b'real_discount_rate', discount_rate )
        ssc.data_set_number( data, b'module_type', 0 )
        ssc.data_set_number( data, b'market', 0 )
        ssc.data_set_number( data, b'ibi_uti_percent', 0 )
        ssc.data_set_number( data, b'stories', 2 )
        ssc.data_set_number( data, b'ibi_oth_percent_deprbas_sta', 0 )
        ssc.data_set_number( data, b'ibi_fed_percent', 0 )
        ssc.data_set_number( data, b'en_cool', 1 )
        ssc.data_set_number( data, b'cbi_fed_amount', 0 )
        ssc.data_set_number( data, b'ibi_uti_amount', 0 )
        ssc.data_set_number( data, b'retrofits', 0 )
        ssc.data_set_number( data, b'en_range', 1 )
        ssc.data_set_number( data, b'gcr', 0.40000000596046448 )
        ssc.data_set_number( data, b'cbi_fed_deprbas_sta', 0 )
        ssc.data_set_number( data, b'itc_sta_percent_maxvalue', 9.9999996802856925e+37 )
        ssc.data_set_number( data, b'ibi_oth_percent_tax_sta', 1 )
        ssc.data_set_number( data, b'ibi_sta_percent', 0 )
        ssc.data_set_number( data, b'cbi_oth_tax_sta', 1 )
        module = ssc.module_create(b'pvwattsv5')	
        ssc.module_exec_set_print( 0 );
        if ssc.module_exec(module, data) == 0:
            print ('pvwattsv5 simulation error')
            idx = 1
            msg = ssc.module_log(module, 0)
            while (msg != None):
                print ('	: ' + msg.decode("utf - 8"))
                msg = ssc.module_log(module, idx)
                idx = idx + 1
            SystemExit( "Simulation Error" );
        ssc.module_free(module)
        module = ssc.module_create(b'belpe')	
        ssc.module_exec_set_print( 0 );
        if ssc.module_exec(module, data) == 0:
            print ('belpe simulation error')
            idx = 1
            msg = ssc.module_log(module, 0)
            while (msg != None):
                print ('	: ' + msg.decode("utf - 8"))
                msg = ssc.module_log(module, idx)
                idx = idx + 1
            SystemExit( "Simulation Error" );
        ssc.module_free(module)
        module = ssc.module_create(b'utilityrate4')	
        ssc.module_exec_set_print( 0 );
        if ssc.module_exec(module, data) == 0:
            print ('utilityrate4 simulation error')
            idx = 1
            msg = ssc.module_log(module, 0)
            while (msg != None):
                print ('	: ' + msg.decode("utf - 8"))
                msg = ssc.module_log(module, idx)
                idx = idx + 1
            SystemExit( "Simulation Error" );
        ssc.module_free(module)
        module = ssc.module_create(b'cashloan')	
        ssc.module_exec_set_print( 0 );
        if ssc.module_exec(module, data) == 0:
            print ('cashloan simulation error')
            idx = 1
            msg = ssc.module_log(module, 0)
            while (msg != None):
                print ('	: ' + msg.decode("utf - 8"))
                msg = ssc.module_log(module, idx)
                idx = idx + 1
            SystemExit( "Simulation Error" );
        ssc.module_free(module)
        annual_energy = ssc.data_get_number(data, b'annual_energy');
        #print ('Annual energy (year 1) = ', annual_energy)
        capacity_factor = ssc.data_get_number(data, b'capacity_factor');
        #print ('Capacity factor (year 1) = ', capacity_factor)
        kwh_per_kw = ssc.data_get_number(data, b'kwh_per_kw');
        #print ('Energy yield (year 1) = ', kwh_per_kw)
        lcoe_nom = ssc.data_get_number(data, b'lcoe_nom');
        #print ('Levelized COE (nominal) = ', lcoe_nom)
        lcoe_real = ssc.data_get_number(data, b'lcoe_real');
        #print ('Levelized COE (real) = ', lcoe_real)
        elec_cost_without_system_year1 = ssc.data_get_number(data, b'elec_cost_without_system_year1');
        #print ('Electricity bill without system (year 1) = ', elec_cost_without_system_year1)
        elec_cost_with_system_year1 = ssc.data_get_number(data, b'elec_cost_with_system_year1');
        #print ('Electricity bill with system (year 1) = ', elec_cost_with_system_year1)
        savings_year1 = ssc.data_get_number(data, b'savings_year1');
        #print ('Net savings with system (year 1) = ', savings_year1)
        npv = ssc.data_get_number(data, b'npv');
        #print ('Net present value = ', npv)
        payback = ssc.data_get_number(data, b'payback');
        #print ('Payback period = ', payback)
        adjusted_installed_cost = ssc.data_get_number(data, b'adjusted_installed_cost');
        #print ('Net capital cost = ', adjusted_installed_cost)
        first_cost = ssc.data_get_number(data, b'first_cost');
        #print ('Equity = ', first_cost)
        loan_amount = ssc.data_get_number(data, b'loan_amount');
        #print ('Debt = ', loan_amount)
        cash_flow = ssc.data_get_array(data,b'cf_after_tax_cash_flow');
        #print ('Cash flow = ', cash_flow)

        #calculate IRR using NumPy
        IRR = numpy.irr(cash_flow)

        hourly_energy = ssc.data_get_array(data,b'gen');
        hourly_load = ssc.data_get_array(data, b'load');
        hourly_to_grid = ssc.data_get_array(data, b'year1_hourly_e_togrid');
        hourly_from_grid = ssc.data_get_array(data, b'year1_hourly_e_fromgrid');
        hourly_system_to_load = ssc.data_get_array(data, b'year1_hourly_system_to_load');
    ##########################################################################################################
    #output
    return [Customer, \
    Scheme, \
    Sensitivity, \
    system_size, \
    NEM_sale_rate,\
    Region, \
    annual_energy, \
    capacity_factor, \
    kwh_per_kw, \
    lcoe_nom, \
    lcoe_real, \
    elec_cost_without_system_year1, \
    elec_cost_with_system_year1, \
    savings_year1, \
    npv, \
    payback, \
    IRR, \
    adjusted_installed_cost, \
    weather_file];

In [31]:
simulate(Customer, Scheme, Sensitivity, weather_file, Region, system_size)

['Res',
 1,
 1,
 0,
 16,
 4068.09228515625,
 15.479803085327148,
 1356.03076171875,
 12.564873695373535,
 9.62972640991211,
 763.0892333984375,
 484.712890625,
 278.3763427734375,
 2562.30712890625,
 13.478580474853516,
 0.08089117661056111,
 5790.0,
 'C:/Users/Admin/Dropbox/Solar NEM (ERI)/Data manipulation/SAM/THA_weather_data/THA_ARANYAPRATHET_484620_IW2/THA_ARANYAPRATHET_484620_IW2.EPW']